# Forecasting for Banking Using Time Series Methods, Live Classes Python notebook
this notebook...


## Live Class 1

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm

In [8]:
# Load the Excel file with all worksheets
file_path = r"C:\Users\hbourrou\Python\Data\bsi_dataset.xlsx"

# Load the data into a dictionary of dataframes (the None option for sheet_name creates a dictionary of dataframes)
excel_data = pd.read_excel(file_path, sheet_name=None)

# Extract worksheet names and display info to make sure everything worked fine
sheet_names = list(excel_data.keys())
print(f"Found {len(sheet_names)} worksheets: {sheet_names}")

Found 6 worksheets: ['BSI', 'GDP', 'int rate short', 'int rate long', 'Comp ind syst stress', 'HICP']


### Cleaning data
Below we work on the individual dataframes to make them consistent.   
The analysis focuses on Germany, and the monthly variables (interest rates, CISS) are converted to quarterly by averaging observations within quarters.

**Note:** for this type of easy dataset, it would be cleaner to do the treatment directly in Excel (i.e. import here only the relevant variables for Germany)

In [ ]:
# Create individual DataFrames
df1 = excel_data[sheet_names[0]]
df2 = excel_data[sheet_names[1]] 
df3 = excel_data[sheet_names[2]]
df4 = excel_data[sheet_names[3]]
df5 = excel_data[sheet_names[4]]
df6 = excel_data[sheet_names[5]]

# Display basic info about each DataFrame
for i, (name, df) in enumerate(excel_data.items(), 1):
    print(f"\n--- Worksheet {i}: '{name}' ---")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"First few rows:")
    print(df.head(3))
    print(f"Data types:")
    print(df.dtypes)
    print("-" * 50)

# Alternative: Create a dictionary for easier access by sheet name
dataframes = {}
for sheet_name, df in excel_data.items():
    dataframes[sheet_name] = df
    
print(f"\nDataFrames created and stored in 'dataframes' dictionary")
print(f"Access individual sheets like: dataframes['{sheet_names[0]}']")

## Live Class 2